In [0]:
dbutils.widgets.text("bronze_table","")
dbutils.widgets.text("bronze_schema","")
dbutils.widgets.text("source_table_name","")
dbutils.widgets.text("storage_account_name","")
dbutils.widgets.text("loadID","")
dbutils.widgets.text("env","")


In [0]:
bronze_table = dbutils.widgets.get("bronze_table").strip()
bronze_schema = dbutils.widgets.get("bronze_schema").strip()
source_table_name = dbutils.widgets.get("source_table_name").strip()
storage_account_name = dbutils.widgets.get("storage_account_name").strip()
loadID = dbutils.widgets.get("loadID").strip()
env = dbutils.widgets.get("env").strip()

In [0]:
spark.sql(f"use hive_metastore.{env}_bronze")

In [0]:
from pyspark.sql.functions import split
bronze_table = bronze_table.split(".")[1]

In [0]:
input_path = f"abfss://landing@{storage_account_name}.dfs.core.windows.net/target_azuresql_files/{source_table_name}"

abfss://landing@.dfs.core.windows.net/target_azuresql_files/


In [0]:
table_df = spark.read.format("parquet").option("header","True").load(input_path)

In [0]:
# Step 2: Count rows in the source table
source_table_count_rows = table_df.count()

# Step 3: Create or replace a temporary view for SQL queries
table_df.createOrReplaceTempView("table_view")


In [0]:
# Step 4: Insert data into the bronze table
spark.sql(f"""
INSERT INTO {bronze_schema}.{bronze_table}
SELECT *, from_utc_timestamp(current_timestamp(), 'IST') as last_insert_dttm,
'{loadID}' as loadID
FROM table_view
""")


In [0]:
bronze_table_count_rows = spark.sql(f"""
SELECT COUNT(*)
FROM {bronze_schema}.{bronze_table}
WHERE loadID = '{loadID}'
""").collect()[0][0]

In [0]:
dbutils.notebook.exit([source_table_count_rows,bronze_table_count_rows])